In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
from matplotlib import cm
import operator
import matplotlib.ticker as ticker
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import matplotlib.lines as mlines

sns.set_context('poster')
sns.set_style('white')
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [ ]:
path = r'/Users/jenniferwh/Dropbox (Allen Institute)/Mesoscale Connectome Papers in Progress/2018 Plaque'
savepath = r'\\allen\programs\celltypes\workgroups\mousecelltypes\T503_Connectivity_in_Alzheimer_Mice\Jennifer\External meetings\2018 SfN'

In [ ]:
dat = pd.read_csv(os.path.join(path, 'unionizes', 'plaque_unionizes.csv'))
dat = dat[dat['control'] == True]

In [ ]:
dat.head()

In [ ]:
dat['count_per_mm_3'] = dat['sum_plaques']/dat['volume']
dat['avg_plaque_volume'] = dat['avg_plaque_volume']/1000 #to convert to cubic microns

In [ ]:
dataset = pd.read_csv(os.path.join(path, '_final_control_dataset.csv'))

In [ ]:
dataset.head()

In [ ]:
methoxy_isids = dataset[dataset['methoxy-X04'] == '3.3 mg/kg 24 hours before perfusion']['Link to image series']

In [ ]:
len(dataset['Link to image series'].unique()) # Missing one that has not been unionized yet

In [ ]:
len(dat['image_series_id'].unique())

In [ ]:
dat = dat[dat['image_series_id'].isin(dataset['Link to image series'])]

In [ ]:
len(dat['image_series_id'].unique())

In [ ]:
mcc = MouseConnectivityCache(manifest_file = '../connectivity/mouse_connectivity_manifest.json')
st = mcc.get_structure_tree()
summary_structures = st.get_structures_by_set_id([167587189])
summary_structure_ids = [item['id'] for item in summary_structures]
ss_acronyms = [item['acronym'] for item in summary_structures]
ont_order = [item['graph_order'] for item in summary_structures]
ont_order_dict = dict(zip(ss_acronyms, ont_order))
print(len(summary_structure_ids))


In [ ]:
coarse_structures = st.get_structures_by_set_id([687527670])
coarse_structure_ids = [item['id'] for item in coarse_structures]
coarse_structure_acronyms = [item['acronym'] for item in coarse_structures]
ft = st.get_structures_by_acronym(['fiber tracts'])
coarse_structure_ids.append(ft[0]['id'])
coarse_structure_acronyms.append(ft[0]['acronym'])
ont_order = [item['graph_order'] for item in coarse_structures]
ont_order.append(ft[0]['graph_order'])
oo_dict_coarse = dict(zip(coarse_structure_acronyms, ont_order))
print(len(coarse_structure_ids))

In [ ]:
mcc = MouseConnectivityCache(manifest_file = '../connectivity/mouse_connectivity_manifest.json')
structure_tree = mcc.get_structure_tree()
ia_map = structure_tree.get_id_acronym_map()
ai_map = {value:key for key, value in ia_map.iteritems()}

In [ ]:
structures = summary_structure_ids + coarse_structure_ids + [997]

In [ ]:
dat.head()

In [ ]:
dat['structure_id'] = [ia_map[structure_acronym] for structure_acronym in dat['structure_acronym']]

In [ ]:
dat.keys()

## Isocortex

In [ ]:
iso = structure_tree.get_structures_by_acronym(['Isocortex'])[0]
iso = structure_tree.descendant_ids([iso['id']])[0]
iso = [structure for structure in iso if structure in summary_structure_ids]

In [ ]:
plot_order = ['FRP', 'MOp', 'MOs', 'SSp-n', 'SSp-bfd', 'SSp-ll', 'SSp-m', 'SSp-ul', 'SSp-tr', 'SSp-un',
              'SSs', 'GU', 'VISC', 'AUDd', 'AUDp', 'AUDpo', 'AUDv', 'VISal', 'VISam', 'VISl', 'VISp', 'VISpl',
              'VISpm', 'VISli', 'VISpor', 'ACAd', 'ACAv', 'PL', 'ILA', 'ORBl', 'ORBm', 'ORBvl', 'AId', 'AIp',
              'AIv', 'RSPagl', 'RSPd', 'RSPv', 'VISa', 'VISrl', 'TEa', 'PERI', 'ECT']

In [ ]:
def get_sorter_index(structure_acronym):
    structures = structure_tree.get_structures_by_acronym([structure_acronym])[0]
    str_ids = structure_tree.descendant_ids([structures['id']])[0]
    str_ss = [ids for ids in str_ids if ids in summary_structure_ids]
    acronyms = [ai_map[stid] for stid in str_ss]
    plot_order = [structure_tree.get_structures_by_id([stid])[0]['graph_order'] for stid in str_ss]
    sorter = dict(zip(acronyms,plot_order))
    sorted_structures = []
    for key, value in sorted(sorter.iteritems(), key=lambda (k,v): (v,k)):
        sorted_structures.append(key)
    sorterIndex = dict(zip(sorted_structures[::-1],range(len(plot_order))))
    return sorterIndex

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
acronyms = []
densities = []
isids = []
for structure in iso:
    structure_acronym = ai_map[structure]
    for isid in dat['image_series_id'].unique():
        isids.append(isid)
        acronyms.append(structure_acronym)
        density = dat[(dat['structure_id'] == structure) & 
                      (dat['image_series_id'] == isid)]['plaque_density'].values[0]
        densities.append(density)
mdf = pd.DataFrame({'image_series_id': isids,
                    'structure': acronyms, 
                    'Percent Plaque Coverage': [density*100 for density in densities]})
mdf.loc[mdf['image_series_id'].isin(methoxy_isids), 'methoxy-X04'] = 'Methoxy-XO4 injected'
mdf.loc[~mdf['image_series_id'].isin(methoxy_isids), 'methoxy-X04'] = ''

ordered = get_sorter_index('Isocortex')
plot_order = [key for key, value in sorted(ordered.iteritems(), key=lambda (k,v): (v,k))][::-1]

fig, ax = plt.subplots(figsize = (5, 15))
for artist in ax.artists:
    artist.set_facecolor('None')
g = sns.stripplot('Percent Plaque Coverage', 'structure',
                   data = mdf[mdf['methoxy-X04'] == ''], alpha = 0.5, order = plot_order, ax = ax,
                 palette = 'gray', s=10)
g = sns.stripplot('Percent Plaque Coverage', 'structure',  
                   data = mdf[mdf['methoxy-X04'] == 'Methoxy-XO4 injected'], alpha = 0.5, 
                  order = plot_order, ax = ax, color = 'red', s=10)
ax.set_ylabel("")
ax.set_xlabel("Percent False Positive")
plt.tight_layout()
ax.set_title('Isocortex', fontsize = 24)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_xlim([-0.1, 2])
plt.xticks([0, 0.5, 1, 1.5, 2])

plt.savefig(os.path.join(
    savepath, 'plaque percent coverage in isocortex structures control mice.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

## Hippocampus

In [ ]:
hipp = st.get_structures_by_acronym(['HPF'])[0]
hipp_ids = st.descendant_ids([hipp['id']])[0]
hipp_ss = [ids for ids in hipp_ids if ids in summary_structure_ids]
hipp_ss.remove(ia_map['IG'])
hipp_ss.remove(ia_map['FC'])
print(len(hipp_ss))

In [ ]:
acronyms = []
densities = []
isids = []
for structure in hipp_ss:
    structure_acronym = ai_map[structure]
    for isid in dat['image_series_id'].unique():
        isids.append(isid)
        acronyms.append(structure_acronym)
        density = dat[(dat['structure_id'] == structure) & 
                      (dat['image_series_id'] == isid)]['plaque_density'].values[0]
        densities.append(density)
mdf = pd.DataFrame({'image_series_id': isids, 
                    'structure': acronyms, 
                    'Percent Plaque Coverage': [density*100 for density in densities]})
mdf.loc[mdf['image_series_id'].isin(methoxy_isids), 'methoxy-X04'] = 'Methoxy-XO4 injected'
mdf.loc[~mdf['image_series_id'].isin(methoxy_isids), 'methoxy-X04'] = ''
fig, ax = plt.subplots(figsize = (5, 6))
ordered = get_sorter_index('HPF')
plot_order = [key for key, value in sorted(ordered.iteritems(), key=lambda (k,v): (v,k))][::-1]
plot_order.remove('FC')
plot_order.remove('IG')
for artist in ax.artists:
    artist.set_facecolor('None')
g = sns.stripplot('Percent Plaque Coverage', 'structure',
                   data = mdf[mdf['methoxy-X04'] == ''], alpha = 0.5, order = plot_order, ax = ax,
                 palette = 'gray', s=10)
g = sns.stripplot('Percent Plaque Coverage', 'structure',  
                   data = mdf[mdf['methoxy-X04'] == 'Methoxy-XO4 injected'], alpha = 0.5, 
                  order = plot_order, ax = ax, color = 'red', s=10)
ax.set_ylabel("")
ax.set_xlabel("Percent False Positive")
plt.tight_layout()
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_xlim([-0.1, 2])
plt.xticks([0, 0.5, 1, 1.5, 2])
ax.set_title('Hippocampus')
plt.savefig(os.path.join(savepath, 'plaque percent coverage in hipp control mice.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)


## Olfactory regions plotted three ways

In [ ]:
olf = st.get_structures_by_acronym(['OLF'])[0]
olf_ids = st.descendant_ids([olf['id']])[0]
olf_ss = [ids for ids in olf_ids if ids in summary_structure_ids]
print(len(olf_ss))

In [ ]:
acronyms = []
densities = []
isids = []
for structure in olf_ss:
    structure_acronym = ai_map[structure]
    for isid in dat['image_series_id'].unique():
        isids.append(isid)
        acronyms.append(structure_acronym)
        density = dat[(dat['structure_id'] == structure) & 
                      (dat['image_series_id'] == isid)]['plaque_density'].values[0]
        densities.append(density)
mdf = pd.DataFrame({'image_series_id': isids, 
                    'structure': acronyms, 
                    'Percent Plaque Coverage': [density*100 for density in densities]})
mdf.loc[mdf['image_series_id'].isin(methoxy_isids), 'methoxy-X04'] = 'Methoxy-XO4 injected'
mdf.loc[~mdf['image_series_id'].isin(methoxy_isids), 'methoxy-X04'] = ''

ordered = get_sorter_index('OLF')
plot_order = [key for key, value in sorted(ordered.iteritems(), key=lambda (k,v): (v,k))][::-1]

fig, ax = plt.subplots(figsize = (5, 5))
for artist in ax.artists:
    artist.set_facecolor('None')
g = sns.stripplot('Percent Plaque Coverage', 'structure',
                   data = mdf[mdf['methoxy-X04'] == ''], alpha = 0.5, order = plot_order, ax = ax,
                 palette = 'gray', s=10)
g = sns.stripplot('Percent Plaque Coverage', 'structure',  
                   data = mdf[mdf['methoxy-X04'] == 'Methoxy-XO4 injected'], alpha = 0.5, 
                  order = plot_order, ax = ax, color = 'red', s=10)
ax.set_ylabel("")
ax.set_xlabel("Percent False Positive")
plt.tight_layout()
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_xlim([-0.1, 2])
plt.xticks([0, 0.5, 1, 1.5, 2])
ax.set_title('Olfactory Areas')
plt.savefig(os.path.join(savepath, 'plaque percent coverage in olf control mice.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

## CTXsp

In [ ]:
ctxsp = st.get_structures_by_acronym(['CTXsp'])[0]
ctxsp_ids = st.descendant_ids([ctxsp['id']])[0]
ctxsp_ss = [ids for ids in ctxsp_ids if ids in summary_structure_ids]
print(len(ctxsp_ss))

In [ ]:
acronyms = []
densities = []
isids = []
for structure in ctxsp_ss:
    structure_acronym = ai_map[structure]
    for isid in dat['image_series_id'].unique():
        isids.append(isid)
        acronyms.append(structure_acronym)
        density = dat[(dat['structure_id'] == structure) & 
                      (dat['image_series_id'] == isid)]['plaque_density'].values[0]
        densities.append(density)
mdf = pd.DataFrame({'image_series_id': isids, 
                    'structure': acronyms, 
                    'Percent Plaque Coverage': [density*100 for density in densities]})
mdf.loc[mdf['image_series_id'].isin(methoxy_isids), 'methoxy-X04'] = 'Methoxy-XO4 injected'
mdf.loc[~mdf['image_series_id'].isin(methoxy_isids), 'methoxy-X04'] = ''

ordered = get_sorter_index('CTXsp')
plot_order = [key for key, value in sorted(ordered.iteritems(), key=lambda (k,v): (v,k))][::-1]

fig, ax = plt.subplots(figsize = (5, 3.5))
for artist in ax.artists:
    artist.set_facecolor('None')
g = sns.stripplot('Percent Plaque Coverage', 'structure',
                   data = mdf[mdf['methoxy-X04'] == ''], alpha = 0.5, order = plot_order, ax = ax,
                 palette = 'gray', s=10)
g = sns.stripplot('Percent Plaque Coverage', 'structure',  
                   data = mdf[mdf['methoxy-X04'] == 'Methoxy-XO4 injected'], alpha = 0.5, 
                  order = plot_order, ax = ax, color = 'red', s=10)
ax.set_ylabel("")
ax.set_xlabel("Percent False Positive")
plt.tight_layout()
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_xlim([-0.1, 2])
plt.xticks([0, 0.5, 1, 1.5, 2])
ax.set_title('Cortical Subplate')
plt.savefig(os.path.join(savepath, 'plaque percent coverage in ctxsp control mice.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

In [ ]:
th = st.get_structures_by_acronym(['TH'])[0]
th_ids = st.descendant_ids([th['id']])[0]
th_ss = [ids for ids in th_ids if ids in summary_structure_ids]
print(len(th_ss))

In [ ]:
acronyms = []
densities = []
isids = []
for structure in th_ss:
    structure_acronym = ai_map[structure]
    for isid in dat['image_series_id'].unique():
        isids.append(isid)
        acronyms.append(structure_acronym)
        density = dat[(dat['structure_id'] == structure) & 
                      (dat['image_series_id'] == isid)]['plaque_density'].values[0]
        densities.append(density)
mdf = pd.DataFrame({'image_series_id': isids, 
                    'structure': acronyms, 
                    'Percent Plaque Coverage': [density*100 for density in densities]})
mdf.loc[mdf['image_series_id'].isin(methoxy_isids), 'methoxy-X04'] = 'Methoxy-XO4 injected'
mdf.loc[~mdf['image_series_id'].isin(methoxy_isids), 'methoxy-X04'] = ''

ordered = get_sorter_index('TH')
plot_order = [key for key, value in sorted(ordered.iteritems(), key=lambda (k,v): (v,k))][::-1]

fig, ax = plt.subplots(figsize = (8, 2))
g = sns.boxplot('structure', 'Percent Plaque Coverage', color = 'white', data = mdf, 
                order = plot_order, dodge = False, ax = ax, fliersize = 1)
for artist in ax.artists:
    artist.set_facecolor('None')
g = sns.stripplot('structure', 'Percent Plaque Coverage', 
                   data = mdf[mdf['methoxy-X04'] == ''], alpha = 0.3, ax = ax,
                 palette = 'gray')
g = sns.stripplot('structure', 'Percent Plaque Coverage', 
                   data = mdf[mdf['methoxy-X04'] == 'Methoxy-XO4 injected'], alpha = 0.3, 
                  ax = ax, color = 'red')
plt.xticks(rotation = 90)
plt.savefig(os.path.join(savepath, 'plaque percent coverage in thal control mice.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

In [ ]:
mb = st.get_structures_by_acronym(['MBmot'])[0]
mb_ids = st.descendant_ids([mb['id']])[0]
mb_ss = [ids for ids in mb_ids if ids in summary_structure_ids]
print(len(mb_ss))

In [ ]:
[ai_map[structure] for structure in mb_ss]

In [ ]:
acronyms = []
densities = []
isids = []
for structure in mb_ids:
    structure_acronym = ai_map[structure]
    for isid in dat['image_series_id'].unique():
        try:
            density = dat[(dat['structure_id'] == structure) & 
                          (dat['image_series_id'] == isid)]['plaque_density'].values[0]
            densities.append(density)
            isids.append(isid)
            acronyms.append(structure_acronym)
        except:
            continue
mdf = pd.DataFrame({'image_series_id': isids, 
                    'structure': acronyms, 
                    'Percent Plaque Coverage': [density*100 for density in densities]})
mdf.loc[mdf['image_series_id'].isin(methoxy_isids), 'methoxy-X04'] = 'Methoxy-XO4 injected'
mdf.loc[~mdf['image_series_id'].isin(methoxy_isids), 'methoxy-X04'] = ''

ordered = get_sorter_index('MBmot')
plot_order = [key for key, value in sorted(ordered.iteritems(), key=lambda (k,v): (v,k))][::-1]
#plot_order = [structure for structure in plot_order if structure not in ['']]

fig, ax = plt.subplots(figsize = (5, 8))
for artist in ax.artists:
    artist.set_facecolor('None')
g = sns.stripplot('Percent Plaque Coverage', 'structure',
                   data = mdf[mdf['methoxy-X04'] == ''], alpha = 0.5, order = plot_order, ax = ax,
                 palette = 'gray', s=10)
g = sns.stripplot('Percent Plaque Coverage', 'structure',  
                   data = mdf[mdf['methoxy-X04'] == 'Methoxy-XO4 injected'], alpha = 0.5, 
                  order = plot_order, ax = ax, color = 'red', s=10)
ax.set_ylabel("")
ax.set_xlabel("Percent False Positive")
plt.tight_layout()
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_xlim([-0.1, 2])
plt.xticks([0, 0.5, 1, 1.5, 2])
ax.set_title('Midbrain')
plt.savefig(os.path.join(savepath, 'plaque percent coverage in midbrain control mice.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

In [ ]:
colors = ['#8da0cb', '#fc8d62', '#ff0000']

In [ ]:
mouse_lines = []
acronyms = []
densities = []
isids = []
for mouse_line in dataset['Mouse Line'].unique():
    mouse_line_ids = dataset[dataset['Mouse Line'] == mouse_line]['Link to image series']
    for structure in iso:
        structure_acronym = ai_map[structure]
        for isid in mouse_line_ids:
            mouse_lines.append(mouse_line)
            isids.append(isid)
            acronyms.append(structure_acronym)
            density = dat[(dat['structure_id'] == structure) & 
                          (dat['image_series_id'] == isid)]['plaque_density'].values[0]
            densities.append(density)
df = pd.DataFrame({'image_series_id': isids, 
                   'Mouse Line': mouse_lines,
                    'structure': acronyms, 
                    'density': [density*100 for density in densities]})
mdf = pd.melt(df, id_vars = ['image_series_id', 'Mouse Line', 'structure'],
              value_name = 'Percent Plaque Coverage')
fig, ax = plt.subplots(figsize = (8, 2))
g = sns.boxplot('structure', 'Percent Plaque Coverage', color = 'white', data = mdf, 
                order = plot_order, dodge = False, ax = ax, fliersize = 1)
for artist in ax.artists:
    artist.set_facecolor('None')
g = sns.stripplot('structure', 'Percent Plaque Coverage', hue = 'Mouse Line', 
                   order = plot_order, hue_order = ['APP/PS1', 'J20'], palette = colors,
                  data = mdf, dodge = True, alpha = 0.3, ax = ax)
plt.xticks(rotation = 90)
#plt.savefig(os.path.join(savepath, 'plaque percent coverage in isocortex control mice by line.pdf'.format(structure)), 
#            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

In [ ]:
mouse_lines = []
acronyms = []
densities = []
isids = []
for mouse_line in dataset['Mouse Line'].unique():
    mouse_line_ids = dataset[dataset['Mouse Line'] == mouse_line]['Link to image series']
    for structure in hipp_ss:
        structure_acronym = ai_map[structure]
        for isid in mouse_line_ids:
            mouse_lines.append(mouse_line)
            isids.append(isid)
            acronyms.append(structure_acronym)
            density = dat[(dat['structure_id'] == structure) & 
                          (dat['image_series_id'] == isid)]['plaque_density'].values[0]
            densities.append(density)
df = pd.DataFrame({'image_series_id': isids, 
                   'Mouse Line': mouse_lines,
                    'structure': acronyms, 
                    'density': [density*100 for density in densities]})
mdf = pd.melt(df, id_vars = ['image_series_id', 'Mouse Line', 'structure'],
              value_name = 'Percent Plaque Coverage')
fig, ax = plt.subplots(figsize = (8, 4))
g = sns.boxplot('structure', 'Percent Plaque Coverage', color = 'white', data = mdf, 
                dodge = False, ax = ax, fliersize = 1)
for artist in ax.artists:
    artist.set_facecolor('None')
g = sns.stripplot('structure', 'Percent Plaque Coverage', hue = 'Mouse Line', palette = colors,
                   hue_order = ['APP/PS1', 'J20'], 
                  data = mdf, dodge = True, alpha = 0.3, ax = ax)
plt.xticks(rotation = 90)
#plt.savefig(os.path.join(savepath, 'plaque percent coverage in hipp control mice by line.pdf'.format(structure)), 
#            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

In [ ]:
mouse_lines = []
acronyms = []
densities = []
isids = []
for mouse_line in dataset['Mouse Line'].unique():
    mouse_line_ids = dataset[dataset['Mouse Line'] == mouse_line]['Link to image series']
    for structure in olf_ss:
        structure_acronym = ai_map[structure]
        for isid in mouse_line_ids:
            mouse_lines.append(mouse_line)
            isids.append(isid)
            acronyms.append(structure_acronym)
            density = dat[(dat['structure_id'] == structure) & 
                          (dat['image_series_id'] == isid)]['plaque_density'].values[0]
            densities.append(density)
df = pd.DataFrame({'image_series_id': isids, 
                   'Mouse Line': mouse_lines,
                    'structure': acronyms, 
                    'density': [density*100 for density in densities]})
mdf = pd.melt(df, id_vars = ['image_series_id', 'Mouse Line', 'structure'],
              value_name = 'Percent Plaque Coverage')
fig, ax = plt.subplots(figsize = (8, 4))
g = sns.boxplot('structure', 'Percent Plaque Coverage', color = 'white', data = mdf, 
                dodge = False, ax = ax, fliersize = 1)
for artist in ax.artists:
    artist.set_facecolor('None')
g = sns.stripplot('structure', 'Percent Plaque Coverage', hue = 'Mouse Line', palette = colors,
                   hue_order = ['APP/PS1', 'J20'], 
                  data = mdf, dodge = True, alpha = 0.3, ax = ax)
plt.xticks(rotation = 90)
#plt.savefig(os.path.join(savepath, 'plaque percent coverage in olf control mice by line.pdf'.format(structure)), 
#            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=300)

In [ ]:
mouse_lines = []
acronyms = []
densities = []
isids = []
for mouse_line in dataset['Mouse Line'].unique():
    mouse_line_ids = dataset[dataset['Mouse Line'] == mouse_line]['Link to image series']
    for structure in coarse_structure_ids:
        structure_acronym = ai_map[structure]
        for isid in mouse_line_ids:
            mouse_lines.append(mouse_line)
            isids.append(isid)
            acronyms.append(structure_acronym)
            density = dat[(dat['structure_id'] == structure) & 
                          (dat['image_series_id'] == isid)]['plaque_density'].values[0]
            densities.append(density)
df = pd.DataFrame({'image_series_id': isids, 
                   'Mouse Line': mouse_lines,
                    'structure': acronyms, 
                    'density': [density*100 for density in densities]})
mdf = pd.melt(df, id_vars = ['image_series_id', 'Mouse Line', 'structure'],
              value_name = 'Percent Plaque Coverage')
mdf.loc[mdf['image_series_id'].isin(methoxy_isids), 'Mouse Line'] = 'Methoxy-X04 Injected'
fig, ax = plt.subplots(figsize = (8, 4))
g = sns.boxplot('structure', 'Percent Plaque Coverage', color = 'white', data = mdf, 
                dodge = False, ax = ax, fliersize = 1)
for artist in ax.artists:
    artist.set_facecolor('None')
g = sns.stripplot('structure', 'Percent Plaque Coverage', hue = 'Mouse Line', palette = colors,
                   hue_order = ['APP/PS1', 'J20', 'Methoxy-X04 Injected'], 
                  data = mdf, dodge = True, alpha = 0.3, ax = ax)
plt.xticks(rotation = 90)


In [ ]:
mouse_lines = []
acronyms = []
densities = []
isids = []
for mouse_line in dataset['Mouse Line'].unique():
    mouse_line_ids = dataset[dataset['Mouse Line'] == mouse_line]['Link to image series']
    for structure in ctxsp_ss:
        structure_acronym = ai_map[structure]
        for isid in mouse_line_ids:
            mouse_lines.append(mouse_line)
            isids.append(isid)
            acronyms.append(structure_acronym)
            density = dat[(dat['structure_id'] == structure) & 
                          (dat['image_series_id'] == isid)]['plaque_density'].values[0]
            densities.append(density)
df = pd.DataFrame({'image_series_id': isids, 
                   'Mouse Line': mouse_lines,
                    'structure': acronyms, 
                    'density': [density*100 for density in densities]})
mdf = pd.melt(df, id_vars = ['image_series_id', 'Mouse Line', 'structure'],
              value_name = 'Percent Plaque Coverage')
fig, ax = plt.subplots(figsize = (8, 4))
g = sns.boxplot('structure', 'Percent Plaque Coverage', color = 'white', data = mdf, 
                dodge = False, ax = ax, fliersize = 1)
for artist in ax.artists:
    artist.set_facecolor('None')
g = sns.stripplot('structure', 'Percent Plaque Coverage', hue = 'Mouse Line', 
                   hue_order = ['APP/PS1', 'J20'], 
                  data = mdf, dodge = True, alpha = 0.3, ax = ax)
plt.xticks(rotation = 90)

In [ ]:
def get_mean_value_per_structure(group, structure_ids):
    means = []
    isids = dataset[dataset['Mouse Line'] == group]['Link to image series'].values
    for structure_id in structure_ids:
        str_mean = np.mean(dat[(dat['structure_id'] == structure_id) & 
                                 (dat['image_series_id'].isin(isids))]
                             ['plaque_density'])
        means.append(str_mean*100)
    structuredat = dict(zip(structure_ids, means))
    return structuredat, len(isids)

In [ ]:
reference_space =  mcc.get_reference_space()
structure_vals, n = get_mean_value_per_structure('APP/PS1', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 1
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=8
h=8
image = [0,0,0]
index = [8000, 4500, 5000]
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(8, 3), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'APP_PS1 false positive map.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=500)

In [ ]:
reference_space =  mcc.get_reference_space()
structure_vals, n = get_mean_value_per_structure('J20', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 1
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=8
h=8
image = [0,0,0]
index = [8000, 4500, 5000]
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(8, 3), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'J20 false positive map.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=500)

In [ ]:
reference_space =  mcc.get_reference_space()
template = mcc.get_template_volume()[0]
means = []
for structure_id in dat['structure_id'].unique():
    str_mean = np.mean(dat[dat['structure_id'] == structure_id]['plaque_density'])
    means.append(str_mean*100)
structure_vals = dict(zip(dat['structure_id'].unique(), means))
rgb_vals = structure_vals.copy()
scale = 1
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=8
h=8
image = [0,0,0]
index = [7000, 4500, 5000]
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(8, 3), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    print(i)
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    if i==0:
        g = plt.imshow(template[280], alpha = 0.5, cmap = 'gray')
    if i==1:
        g = plt.imshow(np.flip(np.rot90(template[:,:,200]), 0), alpha = 0.5, cmap = 'gray')
    if i==2:
        g = plt.imshow(np.rot90(template[:,180, :]), alpha = 0.5, cmap = 'gray')
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('False Positive % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.2, 2), 
                         np.round(maxval*.4, 2), np.round(maxval*.6, 2),
                        np.round(maxval*.8, 2), np.round(maxval, 2)])
plt.savefig(os.path.join(savepath, 'false positive map.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=1000)

In [ ]:
reference_space =  mcc.get_reference_space()
template = mcc.get_template_volume()[0]
means = []
for structure_id in dat['structure_id'].unique():
    str_mean = np.mean(dat[dat['structure_id'] == structure_id]['plaque_density'])
    means.append(str_mean*100)
structure_vals = dict(zip(dat['structure_id'].unique(), means))
rgb_vals = structure_vals.copy()
scale = 1
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=8
h=8
image = [0,0,0]
index = [5975, 2975, 5725]
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(8, 3), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    print(i)
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    if i==0:
        g = plt.imshow(template[239], alpha = 0.5, cmap = 'gray')
    if i==1:
        g = plt.imshow(np.flip(np.rot90(template[:,:,229]), 0), alpha = 0.5, cmap = 'gray')
    if i==2:
        g = plt.imshow(np.rot90(template[:,119, :]), alpha = 0.5, cmap = 'gray')
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('False Positive % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.2, 2), 
                         np.round(maxval*.4, 2), np.round(maxval*.6, 2),
                        np.round(maxval*.8, 2), np.round(maxval, 2)])
plt.savefig(os.path.join(savepath, 'false positive map 2.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=1000)

In [ ]:
savepath

In [ ]:
template.shape

In [ ]:
reference_space =  mcc.get_reference_space()
template = mcc.get_template_volume()[0]
means = []
for structure_id in dat['structure_id'].unique():
    str_mean = np.mean(dat[dat['structure_id'] == structure_id]['plaque_density'])
    means.append(str_mean*100)
structure_vals = dict(zip(dat['structure_id'].unique(), means))
rgb_vals = structure_vals.copy()
scale = 1
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=8
h=8
image = [0,0,0]
index = [8200, 5200, 7350]
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(8, 3), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    print(i)
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    if i==0:
        g = plt.imshow(template[338], alpha = 0.5, cmap = 'gray')
    if i==1:
        g = plt.imshow(np.flip(np.rot90(template[:,:,294]), 0), alpha = 0.5, cmap = 'gray')
    if i==2:
        g = plt.imshow(np.rot90(template[:,208, :]), alpha = 0.5, cmap = 'gray')
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
plt.savefig(os.path.join(savepath, 'false positive map 3.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=500)

In [ ]:
cdat = dat.groupby('structure_acronym').mean().reset_index().sort_values(by = 'plaque_density', ascending = False)

In [ ]:
cdat.to_csv(os.path.join(savepath, 'control_false_positive_mean_by_structure.csv'))

In [ ]:
cdat_median = dat.groupby('structure_acronym').median().reset_index().sort_values(by = 'plaque_density', ascending = False)

In [ ]:
cdat.to_csv(os.path.join(savepath, 'control_false_positive_median_by_structure.csv'))